In [ ]:
%config Completer.use_jedi = False
import sys, IPython
print("IPython version:", IPython.__version__)
import jedi
print("Jedi version:", jedi.__version__)

IPython version: 9.8.0
Jedi version: 0.19.2


In [22]:
import tiledb
import numpy as np
import shutil
import os
import math
import pandas as pd


In [ ]:
help(tiledb.Dim)

In [29]:
array_name = "my_dense_array"

if os.path.exists(array_name):
    shutil.rmtree(array_name)

# 定义schema
d1 = tiledb.Dim(name="rows",domain=(1, 4), tile=2, dtype=np.int32)
d2 = tiledb.Dim(name="cols",domain=(1, 4), tile=2, dtype=np.int32)
domain = tiledb.Domain(d1, d2)

# 创建属性
attr = tiledb.Attr(name="a", dtype=np.int32)

# 创建数组
schema = tiledb.ArraySchema(domain=domain, attrs=(attr,), sparse=False)
tiledb.Array.create(array_name, schema)

with tiledb.open(array_name, 'r') as A:
    result = A
result

DenseArray(uri='my_dense_array', mode=closed)

In [30]:
data = np.array([
    [1, 2, 3, 4], 
    [5, 6, 7, 8], 
    [9, 10, 11, 12], 
    [13, 14, 15, 16]
], dtype=np.int32)

with tiledb.open(array_name, "w") as A:
    A[:] = data
    

with tiledb.open(array_name, 'r') as A:
    result = A[1:3, 1:3]
    print(result)
    print(A[:])

# with tiledb.open(array_name, 'r') as A:
#     # 获取 Schema 信息
#     dom = A.schema.domain
#     dim_rows = dom.dim(0)
#     dim_cols = dom.dim(1)
    
#     row_tile_extent = dim_rows.tile
#     col_tile_extent = dim_cols.tile
    
#     print(f"整体维度: {dim_rows.domain} x {dim_cols.domain}")
#     print(f"瓦片大小 (Tile Extent): {row_tile_extent} x {col_tile_extent}\n")

#     # 计算总瓦片数
#     # 也就是：(行长度 / 行瓦片大小) * (列长度 / 列瓦片大小)
#     num_tiles_row = math.ceil((dim_rows.domain[1] - dim_rows.domain[0] + 1) / row_tile_extent)
#     num_tiles_col = math.ceil((dim_cols.domain[1] - dim_cols.domain[0] + 1) / col_tile_extent)
#     total_tiles = num_tiles_row * num_tiles_col
    
#     print(f"横向 {num_tiles_col} 个 x 纵向 {num_tiles_row} 个 = 总共 {total_tiles} 个瓦片\n")

    
#     # 创建一个空的 DataFrame 来展示
#     display_grid = np.empty((4, 4), dtype=object)
    
#     for r in range(1, 5):      # Rows 1 to 4
#         for c in range(1, 5):  # Cols 1 to 4
#             # 计算逻辑：(当前坐标 - 起始坐标) / 瓦片跨度
#             tile_row_idx = (r - dim_rows.domain[0]) // row_tile_extent
#             tile_col_idx = (c - dim_cols.domain[0]) // col_tile_extent
            
#             # 组合成瓦片坐标 (0,0), (0,1) 等
#             tile_coord = f"Tile({tile_row_idx},{tile_col_idx})"
            
#             # 填入展示网格 (索引要减1因为numpy是0-based)
#             display_grid[r-1, c-1] = tile_coord

#     # 打印可视化结果
#     df = pd.DataFrame(display_grid, 
#                       index=[f"Row {i}" for i in range(1,5)], 
#                       columns=[f"Col {i}" for i in range(1,5)])
#     print(df)

# type(dim_rows.domain)

OrderedDict({'a': array([[1, 2],
       [5, 6]], dtype=int32)})
OrderedDict({'a': array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32)})


In [19]:
with tiledb.open(array_name, 'r') as A:
        result = A[1:3, 1:3]
        print(result)
        print(A[:])





OrderedDict({'a': array([[1, 2],
       [5, 6]], dtype=int32)})
OrderedDict({'a': array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32)})


In [71]:
sparse_array_name = "my_sparse_array"

if os.path.exists(array_name):
    shutil.rmtree(array_name)

d1 = tiledb.Dim(name="user_id",domain=(1, 1000), tile=10, dtype=np.int32)
dom = tiledb.Domain(d1)

# 属性
attr1 = tiledb.Attr(name="score", dtype=np.float32)

schema  = tiledb.ArraySchema(domain=dom, attrs=(attr1,), sparse=True, capacity=10000)
tiledb.Array.create(sparse_array_name, schema)

schema.capacity

10000

In [72]:
I = [1, 10, 100]           # 坐标 (Indices)
data = [99.5, 88.0, 60.2]  # 值 (Values)

with tiledb.open(sparse_array_name, 'w') as A:
    # 稀疏写入：指定坐标 I，写入数据 data
    A[I] = data

print(f"\n稀疏数组已写入: {sparse_array_name}")


稀疏数组已写入: my_sparse_array


In [73]:
with tiledb.open("my_sparse_array", 'r') as A:
    result = A[1:10]
    print(result)




OrderedDict({'score': array([99.5], dtype=float32), 'user_id': array([1], dtype=int32)})


In [74]:
with tiledb.open("my_sparse_array", 'r') as A:
    # 1. 读取所有非空数据的坐标
    # A[:] 对于稀疏数组会返回所有非空数据
    data_read = A[:]
    coords = data_read["user_id"]
    print(coords)   
    dim = A.schema.domain.dim("user_id")
    tile_extent = dim.tile        # 这里是 10
    domain_start = dim.domain[0]  # 这里是 1

    tile_indices = (coords - domain_start) // tile_extent
    print(type(tile_indices))

    unique_tiles = np.unique(tile_indices)

    print(f"写入的坐标 (Coords): {coords}")
    print(f"对应的瓦片索引 (Tile Indices): {tile_indices}")
    print(f"-" * 30)
    print(f"实际占用的逻辑瓦片数量: {len(unique_tiles)} 个")
    print(f"占用的瓦片 ID: {unique_tiles}")

[  1  10 100]
<class 'numpy.ndarray'>
写入的坐标 (Coords): [  1  10 100]
对应的瓦片索引 (Tile Indices): [0 0 9]
------------------------------
实际占用的逻辑瓦片数量: 2 个
占用的瓦片 ID: [0 9]


In [75]:
array_path = "my_sparse_array"

print(f"📂 文件夹结构: {array_path}")
print("-" * 40)

for root, dirs, files in os.walk(array_path):
    # 计算缩进，为了好看
    level = root.replace(array_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}📁 {os.path.basename(root)}/")
    for f in files:
        print(f"{indent}    📄 {f}")

📂 文件夹结构: my_sparse_array
----------------------------------------
📁 my_sparse_array/
    📁 __labels/
    📁 __fragment_meta/
    📁 __fragments/
        📁 __1765720750914_1765720750914_3b27f1424db185340f802b15dc703ecb_22/
            📄 __fragment_metadata.tdb
            📄 a0.tdb
            📄 d0.tdb
    📁 __schema/
        📄 __1765720750390_1765720750390_1d3add5694b2353cb54743ede07bfbaf
        📁 __enumerations/
    📁 __commits/
        📄 __1765720750914_1765720750914_3b27f1424db185340f802b15dc703ecb_22.wrt
    📁 __meta/


In [76]:
array_name = "my_sparse_array"
# 这告诉 TileDB：请把底层的物理块边界信息（MBR）也加载出来给我看
fragments = tiledb.array_fragments(array_name, include_mbrs=True)
print(f"总共有 {len(fragments)} 个物理片段 (Fragments)")

for i, frag in enumerate(fragments):

    print(f"物理 URI: {frag.uri.split('/')[-1]}")
    # 2版本差异处理：有些版本可能叫 non_empty_domain，有些在 mbrs 里体现
    # 我们直接看最重要的 MBRs (Minimum Bounding Rectangles)
    # MBR 就是物理上的“瓦片/数据块”的边界
    if hasattr(frag, 'mbrs'):
        print(f"   物理索引块 (MBRs/Tiles): {len(frag.mbrs)} 个")
        for m_idx, mbr in enumerate(frag.mbrs):
            # mbr 通常是一个元组，表示 ((dim1_min, dim1_max), ...)
            print(f"     物理块 #{m_idx} 范围: {mbr}")
            # 验证你的猜想：
            # 如果是块 #0，范围应该是 (1, 10)
            # 如果是块 #1，范围应该是 (100, 100)
    else:
        print("   (当前对象未包含 mbrs 属性，请检查 tiledb 版本)")

总共有 1 个物理片段 (Fragments)
物理 URI: __1765720750914_1765720750914_3b27f1424db185340f802b15dc703ecb_22
   物理索引块 (MBRs/Tiles): 1 个
     物理块 #0 范围: ((1, 100),)


In [70]:
schema.capacity

2

In [77]:
import scanpy as sc
import pandas as pd
import anndata

In [78]:
datapath = "/fast/data/scTFM/ae/SRX25319037.h5ad"

In [80]:
adata = sc.read_h5ad(datapath)
adata

AnnData object with n_obs × n_vars = 5166 × 36601
    obs: 'gene_count', 'umi_count', 'SRX_accession'
    var: 'gene_symbols', 'feature_types'

In [108]:
n_cells = 100
n_genes = 50
X = np.random.randint(0, 10, size=(n_cells, n_genes))

In [109]:
obs_data = {
    "cell_id": [f"cell_{i}" for i in range(n_cells)],
    "cell_type": np.random.choice(["T-Cell", "B-Cell", "Macrophage"], n_cells),
    "tissue": np.random.choice(["Lung", "Liver", "Blood"], n_cells),
    "is_primary": np.random.choice([True, False], n_cells),
}
obs = pd.DataFrame(obs_data).set_index("cell_id")

obs


,cell_type,tissue,is_primary
cell_id,,,
cell_0,T-Cell,Blood,True
cell_1,Macrophage,Liver,True
cell_2,B-Cell,Lung,True
cell_3,Macrophage,Blood,False
cell_4,T-Cell,Liver,False
...,...,...,...
cell_95,Macrophage,Liver,False
cell_96,B-Cell,Liver,True
cell_97,T-Cell,Liver,True


In [110]:
var_data = {
    "gene_id": [f"ENSG000{i}" for i in range(n_genes)],
    "gene_symbol": [f"Gene_{i}" for i in range(n_genes)]
}
var = pd.DataFrame(var_data).set_index("gene_id")

In [112]:
adata = anndata.AnnData(X=X, obs=obs, var=var)
adata




adata.write("sparse_sim.h5ad")

In [99]:
raw_path = "raw_tiledb_group"
if os.path.exists(raw_path): shutil.rmtree(raw_path)


tiledb.group_create(raw_path)

In [100]:
import scipy.sparse

tiledb.from_pandas(
    uri=f"{raw_path}/obs", 
    dataframe=adata.obs, 
    sparse=True  # 对于字符串索引，推荐稀疏
)

tiledb.from_pandas(
    uri=f"{raw_path}/var", 
    dataframe=adata.var, 
    sparse=True
)

x_uri = f"{raw_path}/X"

d1 = tiledb.Dim(name="cell_idx", domain=(0, n_cells-1), tile=10, dtype=np.int32)
d2 = tiledb.Dim(name="gene_idx", domain=(0, n_genes-1), tile=10, dtype=np.int32)
dom = tiledb.Domain(d1, d2)

attr = tiledb.Attr(name="value", dtype=adata.X.dtype)

# 创建 Schema
schema = tiledb.ArraySchema(domain=dom, sparse=True, attrs=[attr]) # 这里为了简单用 Dense
tiledb.Array.create(x_uri, schema)

if scipy.sparse.issparse(adata.X):
    x_coo = adata.X.tocoo()
else:
    # 把密集数组直接包成 COO 格式
    x_coo = scipy.sparse.coo_matrix(adata.X)
# TileDB 稀疏写入需要：坐标列表 (I, J) 和 值列表 (V)
# 我们把 scipy 的 csr_matrix 转成 coo_matrix，方便提取 row, col, data

rows = x_coo.row  # 所有的行坐标 [0, 0, 1, 5, ...]
cols = x_coo.col  # 所有的列坐标 [1, 5, 2, 9, ...]
vals = x_coo.data # 对应的数值   [10, 5, 3, 8, ...]



/gpfs/flash/home/jcw/anaconda3/envs/sctfm/lib/python3.12/site-packages/tiledb/dataframe_.py:41: UserWarning: PyArrow version >= 1.0 is suggested for dataframe functionality.
                  Please `pip install pyarrow>=1.0`.
  warnings.warn(pa_error)
/gpfs/flash/home/jcw/anaconda3/envs/sctfm/lib/python3.12/site-packages/tiledb/dataframe_.py:41: UserWarning: PyArrow version >= 1.0 is suggested for dataframe functionality.
                  Please `pip install pyarrow>=1.0`.
  warnings.warn(pa_error)
/gpfs/flash/home/jcw/anaconda3/envs/sctfm/lib/python3.12/site-packages/tiledb/dataframe_.py:41: UserWarning: PyArrow version >= 1.0 is suggested for dataframe functionality.
                  Please `pip install pyarrow>=1.0`.
  warnings.warn(pa_error)


In [104]:
len(vals)

with tiledb.open(x_uri, 'w') as A:
    # 稀疏写入语法：A[行坐标, 列坐标] = 值
    A[rows, cols] = vals

with tiledb.Group(raw_path, 'w') as G:
    G.add("obs", name="obs")
    G.add("var", name="var")
    G.add("X", name="X")

print(f"[Raw TileDB] 构建完成！")

[Raw TileDB] 构建完成！


In [106]:
with tiledb.open(x_uri, 'r') as A:
    print("\n[Raw TileDB] 验证读取:")
    print(f"Schema Sparse? {A.schema.sparse}")
    
    # 读取坐标 (0, 0) 到 (5, 5) 范围内的非零值
    # 注意：稀疏数组返回的是 OrderedDict，包含坐标和属性
    res = A[0:10, 0:10]
    
    print("读取左上角 5x5 区域的非零值:")
    df_res = pd.DataFrame(res)
    if not df_res.empty:
        print(df_res)
    else:
        print("该区域全是 0 (没有存储任何数据)")


[Raw TileDB] 验证读取:
Schema Sparse? True
读取左上角 5x5 区域的非零值:
    value  cell_idx  gene_idx
0       6         0         0
1       9         0         1
2       3         0         2
3       8         0         3
4       8         0         4
..    ...       ...       ...
85      6         9         4
86      7         9         5
87      8         9         6
88      1         9         7
89      2         9         9

[90 rows x 3 columns]


In [114]:
import tiledbsoma
import tiledbsoma.io
import anndata
import scipy.sparse
import numpy as np
import pandas as pd
import shutil
import os
import tiledb  # 仅用于最后展示底层验证

# ==========================================
# 第一步：生成模拟的稀疏 .h5ad 文件
# ==========================================
print("1. 正在生成模拟稀疏数据...")
n_cells = 100
n_genes = 50

# 生成真正的稀疏矩阵 (CSR 格式)
# density=0.2 表示只有 20% 的非零数据
X_sparse = scipy.sparse.random(n_cells, n_genes, density=0.2, format='csr')
X_sparse.data = (X_sparse.data * 10).astype(np.int32) # 模拟 count 值

# 生成元数据
obs = pd.DataFrame(index=[f"cell_{i}" for i in range(n_cells)], 
                   data={"cell_type": ["T-Cell"]*n_cells, "tissue": ["Lung"]*n_cells})
var = pd.DataFrame(index=[f"gene_{i}" for i in range(n_genes)], 
                   data={"gene_symbol": [f"G{i}" for i in range(n_genes)]})

# 保存文件
h5ad_file = "sparse_sim.h5ad"
adata = anndata.AnnData(X=X_sparse, obs=obs, var=var)
adata.write(h5ad_file)
print(f"   已保存: {h5ad_file}")


# ==========================================
# 第二步：转换为 TileDB SOMA
# ==========================================
soma_path = "soma_sparse_experiment"
if os.path.exists(soma_path): shutil.rmtree(soma_path)

print("\n2. 正在转换为 TileDB SOMA...")
tiledbsoma.io.from_h5ad(
    experiment_uri=soma_path,
    input_path=h5ad_file,
    measurement_name="RNA"
)
print(f"   转换完成: {soma_path}")


# ==========================================
# 第三步：验证与读取 (修复了报错部分)
# ==========================================
print("\n3. 验证数据结构...")
with tiledbsoma.Experiment.open(soma_path) as exp:
    # 获取 X 数组对象
    x_array = exp.ms['RNA'].X['data']
    
    # --- 【核心修复】 ---
    # SOMA 对象的 .schema 返回的是 Arrow Schema，没有 sparse 属性
    # 我们应该检查它的 soma_type 或者使用 isinstance
    print(f"   [验证 A] SOMA 对象类型: {x_array.soma_type}")
    # 预期输出: SOMASparseNDArray (这就证明它是稀疏的)
    
    if isinstance(x_array, tiledbsoma.SparseNDArray):
        print("   [验证 B] 确认: 这是一个稀疏数组 (SparseNDArray)")

    # --- 读取演示 ---
    # 读取第 0 个细胞的数据
    # coords=(0, slice(None)) 表示: 选取第0行(细胞)，选取所有列(基因)
    data = x_array.read(coords=([0], slice(None))).tables().concat()
    
    print("\n   [数据读取] 第 0 个细胞的非零表达量:")
    print(data.to_pandas())


# ==========================================
# 附加：如果你非要看 .schema.sparse (True)
# ==========================================
print("\n4. (可选) 底层物理属性验证...")
# 我们获取 SOMA 数组的物理路径，用原生 tiledb 库打开
x_uri = os.path.join(soma_path, "ms", "RNA", "X", "data")

with tiledb.open(x_uri, 'r') as A:
    print(f"   底层 TileDB Schema Sparse 属性: {A.schema.sparse}")
    # 这里一定会输出 True

1. 正在生成模拟稀疏数据...
   已保存: sparse_sim.h5ad

2. 正在转换为 TileDB SOMA...
   转换完成: soma_sparse_experiment

3. 验证数据结构...
   [验证 A] SOMA 对象类型: SOMASparseNDArray
   [验证 B] 确认: 这是一个稀疏数组 (SparseNDArray)

   [数据读取] 第 0 个细胞的非零表达量:
    soma_dim_0  soma_dim_1  soma_data
0            0          11          5
1            0          14          1
2            0          18          0
3            0          22          6
4            0          23          2
5            0          26          7
6            0          27          7
7            0          32          1
8            0          39          0
9            0          40          3
10           0          42          1
11           0          45          5

4. (可选) 底层物理属性验证...
   底层 TileDB Schema Sparse 属性: True
